<a href="https://colab.research.google.com/github/kkech/RAG_Workshop/blob/main/Google_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval-Augmented Generation (RAG)

Retrieval-Augmented Generation (RAG) is a technique that enhances the performance of language models by integrating information retrieval mechanisms with generative capabilities. This approach allows the model to generate more accurate and contextually relevant responses by leveraging external knowledge sources.

![RAG Schema](https://miro.medium.com/v2/resize:fit:720/format:webp/1*jy3OIYsIi9NcsDsfaNC_6w.png)


## How RAG Works

The RAG process can be broken down into the following steps, as illustrated in the schema:

1. **Documents Preparation:**
   - **Chunked Texts:** The input documents are first divided into smaller, manageable chunks of text. This step ensures that the retrieval process can handle large documents effectively.

2. **Embeddings Generation:**
   - **Generate Embeddings:** Each chunked text is then converted into embeddings using a suitable embedding model. These embeddings capture the semantic meaning of the text and are essential for the retrieval process.

3. **Vector Database:**
   - **Store Embeddings:** The generated embeddings are stored in a vector database. This database allows efficient similarity search to find relevant text passages based on the input prompt.

4. **Prompt Handling:**
   - **Prompt Embedding:** When a prompt is received, it is also converted into an embedding using the same or a similar embedding model used for the documents.

5. **Retrieval Phase:**
   - **Retrieve Relevant Passages:** The prompt embedding is used to search the vector database for the most relevant text passages. These passages provide the context needed for generating a response.

6. **Prompt + Context:**
   - **Combine Prompt and Context:** The retrieved relevant text passages (context) are combined with the original prompt. This combined input is then fed into the language model.

7. **Generation Phase:**
   - **LLM (Large Language Model):** The language model processes the combined prompt and context to generate a response. The integration of retrieved context helps the model produce more accurate and informed outputs.

8. **Result:**
   - **Output:** The generated response is returned as the final result, incorporating the relevant information retrieved from the external knowledge sources.

## Benefits of RAG

- **Improved Accuracy:** By incorporating relevant external information, RAG enhances the accuracy of the generated responses.
- **Contextual Relevance:** The retrieval phase ensures that the generated text is contextually relevant, addressing specific queries more effectively.
- **Knowledge Integration:** RAG allows the integration of up-to-date information from external sources, making it suitable for dynamic and information-rich tasks.

RAG provides a powerful framework for combining the strengths of information retrieval and text generation, enabling the creation of more robust and context-aware language models.


Retrieval Augmented Generation (RAG)
![RAG](https://drive.google.com/uc?id=1Y6IT6wKrDwjOIX-3IZlq5LVlHRcEHebC)

In [ ]:
!pip install datasets pymongo sentence_transformers gradio

Load Dataset
![Dataset loading](https://drive.google.com/uc?id=1fC7fCFRxfgzC1sfemaJEK5fGlk3yqqpY)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("ashmib/wikivoyage-eu-city-embeddings", download_mode="force_redownload") ## downloading it from HuggingFace
dataset.set_format(type='pandas') ## converting it into pandas
df = dataset["train"][:]
df.head()

Embed Model
![Embedding.png](https://drive.google.com/uc?id=1yMntmFJdugi1ib-JNOoqlo_WbPgWvDy3)

In [ ]:
from sentence_transformers import SentenceTransformer

def get_embedding(text: str) -> list[float]:

    embedding_model = SentenceTransformer("thenlper/gte-large")
    if not text.strip():
        print("Attempted to get embedding for empty text.")
        return []

    embedding = embedding_model.encode(text)

    return embedding.tolist()

Data Ingestion & Vector Database Setup

In [ ]:
import pymongo
import os
from google.colab import userdata

def get_mongo_url():
    username = userdata.get("MONGO_USERNAME")
    password = userdata.get("MONGO_PW")
    # uri_string = "@cluster0.62unmco.mongodb.net/"
    uri_string = userdata.get("MONGO_CONN_STR")
    mongo_url = f"mongodb+srv://{username}:{password}{uri_string}"
    return mongo_url


def get_mongo_client(mongo_url):
    """Establish connection to the MongoDB."""
    if not mongo_url:
        print("MONGO_URI not set in environment variables")
    try:
        client = pymongo.MongoClient(mongo_url)
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None

## Data ingestion in Mongodb

mongo_url = get_mongo_url()
if not mongo_url:
    print("MONGO_creds not set in environment variables")

# establishes database connection
mongo_client = get_mongo_client(mongo_url)
# creates database
db = mongo_client["wikivoyage_cities"]
# creates collection
collection = db["wikivoyage_collection_2"]

# Delete any existing records in the collection
collection.delete_many({})

# data ingestion into mongoDB
documents = df.to_dict('records')
collection.insert_many(documents)
print("Data ingestion into MongoDB completed")

Data Retrieval based on context
![Retrieval](https://drive.google.com/uc?id=1Qnk-v5ZGFo1DN3Ye6iylcL4hOL9ATHjR)

In [ ]:
def query_results(query, mongo_url):
    mongo_client = get_mongo_client(mongo_url)
    db = mongo_client["wikivoyage_cities"]

    query_embedding = get_embedding(query)
    results = db.wikivoyage_collection_2.aggregate([
        {
            "$vectorSearch": {
                "index": "vector_index",
                "path": "embedding",
                "queryVector": query_embedding,
                "numCandidates": 150,
                "limit": 5
            }
        }
    ])
    return results

def get_search_result(query, mongo_url):
    get_knowledge = query_results(query, mongo_url)
    print(get_knowledge)

    search_result = ""
    for result in get_knowledge:
        search_result += f"City: {result.get('city', 'N/A')}, Abstract: {result.get('combined', 'N/A')}\n"

    return search_result

Augmentation and Generation
![Augmentation & Generation](https://drive.google.com/uc?id=11-Hn2WbD8_KqsQpxny9ovH1IZa78r7PW)


In [ ]:
from huggingface_hub import InferenceClient

HF_token = userdata.get("HF_TOKEN")

def generate_text(query, model_name: str | None = "google/gemma-2b-it"):
    if model_name is None:
        model_name = "google/gemma-2b-it"

    # establish mongo connection
    mongo_url = get_mongo_url()

    # get the top 5 most similar documents from the Vector data base
    source_information = get_search_result(query, mongo_url)

    # augment the query with the context
    combined_information = (
        f"Query: {query}\nContinue to answer the query by using the Search Results:\n{source_information}."
    )

    # use the HF inference client to generate the text
    client = InferenceClient(model_name, token=HF_token)
    stream = client.text_generation(prompt=combined_information, details=True, stream=True, max_new_tokens=2048,
                                    return_full_text=False)
   # formatting the output
    output = ""

    for response in stream:
        output += response.token.text

    if "<eos>" in output:
        output = output.split("<eos>")[0]
    return output

query = "I am planning a vacation to Spain. Can you suggest a one-week itinerary including must-visit places and local cuisines to try?"
generate_text(query)

Creating UI using [Gradio](https://www.gradio.app/docs)

In [ ]:
import gradio as gr
examples = [["I'm planning a vacation to France. Can you suggest a one-week itinerary including must-visit places and "
             "local cuisines to try?", None],
            ["Recommend places that are similar to Istanbul in terms of architecture", None],
            ]

demo = gr.Interface(
    fn=generate_text,
    inputs=["text",
            gr.Dropdown(
                ["google/gemma-2b-it",], label="Models", info="Will "
                                                                                                             "add "
                                                                                                             "more "
                                                                                                             "models "
                                                                                                             "later! "
            ),
            ],
    title="🇪🇺 Euro City TravelBot 🇪🇺",
    description="Travel related queries for Europe.",
    outputs=["text"],
    examples=examples,
)
demo.launch()